# Звіт до індивідуального завдання №1 «Обчислення математичних виразів»

**Дисципліна:** «Комп'ютерна техніка, алгоритмізація та програмування»  
**Студент:** ____________________  
**Група:** ____________________  
**Варіант:** 6

## Мета роботи

Сформувати практичні навички побудови лінійних та розгалужених алгоритмів мовою Python, опанувати застосування елементарних функцій модуля `math`, перевірку області допустимих значень (ОДЗ) математичного виразу та забезпечити коректне відображення результатів обчислень із заданою точністю.

## Постановка задачі

Необхідно розробити програму, що запитує дійсне значення `x`, перевіряє виконання умов ОДЗ і обчислює функцію згідно з варіантом 6 додатку А методичних вказівок:

$$
Y = \frac{x^{-x \sin^2 x} + \ln(6{,}23\,x^2)}{\sqrt[3]{x^2 - 24{,}8} - \operatorname{ctg}\left(\frac{2x}{3}\right)}.
$$

Для забезпечення визначеності виразу необхідно виконати такі умови (з урахуванням допуску $t = 10^{-12}$):

- **Основна вимога:** `x > 0` (для дійсного обчислення степеня у виразі `x ** (-x·sin^2 x)`).
- **Логарифм:** $6.23·x^2 > eps\_zero$.
- **Котангенс:** `|tan(2x/3)| > eps_tan`, адже `ctg(2x/3) = 1 / tan(2x/3)`.
- **Знаменник дробу:** $|\sqrt[3]{x^2 - 24{,}8} - ctg(2x/3)| > eps\_zero$, щоб запобігти діленню на нуль.

Примітка: у формулах наведено десяткові коми (наприклад, `6{,}23`), тоді як у коді Python використовується десяткова крапка (`6.23`, `24.8`).

Результатом роботи має бути відображення обраного варіанту, проміжних параметрів (за потреби), введеного значення `x` та обчисленого значення `Y`. У випадку порушення будь-якої з умов ОДЗ програма повинна повідомити про причину зупинки обчислень.

## Блок-схема алгоритму

```mermaid
flowchart TD
    A(["Початок"]) --> B["Введення x"]
    B --> C{"x > 0"}
    C -- Ні --> C1["порушено ОДЗ: x має бути додатним"]
    C1 --> X(["Кінець"])
    C -- Так --> D["log_arg = 6.23 · x^2"]
    D --> E{"log_arg > eps_zero"}
    E -- Ні --> E1["порушено ОДЗ: аргумент логарифма має бути додатним"]
    E1 --> X
    E -- Так --> F["radicand = x^2 - 24.8"]
    F --> G["cube_root = cbrt(radicand)"]
    G --> H["angle = 2x/3"]
    H --> I["tan_value = tan(angle)"]
    I --> J{"abs(tan_value) > eps_tan"}
    J -- Ні --> J1["порушено ОДЗ: котангенс не визначений"]
    J1 --> X
    J -- Так --> K["cotangent = 1 / tan_value"]
    K --> L["denominator = cube_root - cotangent"]
    L --> M{"abs(denominator) > eps_zero"}
    M -- Ні --> M1["порушено ОДЗ: знаменник дорівнює нулю"]
    M1 --> X
    M -- Так --> N["power_term = x ** (-x · sin^2 x)"]
    N --> O["numerator = power_term + ln(log_arg)"]
    O --> P["y = numerator / denominator"]
    P --> Q["Вивести x та y"]
    Q --> X
```

## Лістинг програми

In [1]:
# Імпортуємо модуль math, щоб користуватися тригонометричними й логарифмічними функціями
import math

# Окремі допуски для чисельної стабільності
eps_zero = 1e-12  # перевірки на нуль (логарифм, знаменник)
eps_tan = 1e-7    # визначеність котангенса

# Зчитуємо введене користувачем значення x та перетворюємо його у тип float
x = float(input("Введіть x: "))

# Перевіряємо першу умову ОДЗ: x має бути додатним (вимагається степеневою частиною)
if x <= 0.0:
    print("порушено ОДЗ: x має бути додатним")
else:
    # Обчислюємо аргумент логарифма 6.23 * x^2
    log_arg = 6.23 * x ** 2
    # Друга перевірка: аргумент натурального логарифма повинен бути додатним
    if log_arg <= eps_zero:
        print("порушено ОДЗ: аргумент логарифма має бути додатним")
    else:
        # Обчислюємо підкореневий вираз для кубічного кореня
        radicand = x ** 2 - 24.8
        # Знаходимо кубічний корінь із урахуванням знака (визначений для всіх дійсних значень)
        cube_root = math.copysign(abs(radicand) ** (1.0 / 3.0), radicand)
        # Готуємо кут для тригонометричних функцій (2x/3)
        angle = 2.0 * x / 3.0
        # Обчислюємо тангенс цього кута для перевірки існування котангенса
        tan_value = math.tan(angle)
        # Третя перевірка: котангенс існує лише коли tan(angle) достатньо віддалений від нуля
        if abs(tan_value) <= eps_tan:
            print("порушено ОДЗ: tan(2x/3) дорівнює нулю, котангенс не визначений")
        else:
            # Обчислюємо котангенс як обернений тангенс
            cotangent = 1.0 / tan_value
            # Формуємо знаменник дробу: кубічний корінь мінус котангенс
            denominator = cube_root - cotangent
            # Четверта перевірка: знаменник не повинен дорівнювати нулю
            if abs(denominator) <= eps_zero:
                print("порушено ОДЗ: знаменник дорівнює нулю")
            else:
                # Обчислюємо степеневий компонент x^(-x * sin^2 x)
                power_term = x ** (-x * (math.sin(x) ** 2))
                # Формуємо чисельник виразу як суму степеневого терма і логарифма
                numerator = power_term + math.log(log_arg)
                # Завершуємо обчислення: ділимо чисельник на сформований знаменник
                y = numerator / denominator
                # Виводимо значення аргументу x для зручності
                print(f"x = {x}")
                # Виводимо підсумковий результат обчислення y з фіксованою точністю
                print(f"y = {y:.6f}")

Введіть x:  2


x = 2.0
y = -1.180919


Скрипт послідовно перевіряє всі вимоги до області допустимих значень: `x > 0`, додатність аргументу
логарифма, існування котангенса (з урахуванням допуску `eps_tan`) та ненульовість різниці між кубічним
коренем і котангенсом у знаменнику.
Після успішного проходу перевірок програма обчислює степеневу частину, додає логарифм, формує знаменник
(`cuberoot - ctg`) і ділить чисельник на цей знаменник. У результаті студент отримує значення `y` разом із
введеним `x`.

## Результати роботи програми

Тестування виконувалося для значень `x`, що задовольняють усі умови ОДЗ. Результати наведено з відносною похибкою не більше $10^{-3}$.

| x | y | Коментар |
|---|---|---|
| 5.0 | -1.106 | ОДЗ дотримано, знаменник додатний |
| 6.0 | 4.255 | ОДЗ дотримано, котангенс визначений |
| 4.712389 | не визначено | tan(2x/3) дорівнює нулю, котангенс не визначений |

У випадку введення значення, що порушує ОДЗ (наприклад, `x ≈ 4.712389`, для якого `tan(2x/3) = 0`), програма виводить діагностичне повідомлення без виконання обчислень функції.

## Висновки

Розроблений скрипт демонструє послідовне застосування умовних конструкцій Python для контролю ОДЗ складного математичного виразу. Перевірка логарифмічної, степеневої та тригонометричної частин дозволяє уникнути ділення на нуль і розрахунків поза визначеною областю. Проведені тестові приклади підтвердили коректність алгоритму та доцільність використання допуску $t = 10^{-12}$ для чисельної стабільності.